In [ ]:
#Run this code cell if you are unable to scroll in RISE on local machine

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'scroll': True,
})

<img src="2.jpeg" style="height: 50%"/>

<img src="3.jpeg" style="height: 50%;"/>

<img src="4.jpeg" style="height: 50%;"/>

<img src="5.jpeg" style="height: 50%;"/>

<img src="6.jpeg" style="height: 50%;"/>

<img src="7.jpeg" style="height: 50%;"/>

<img src="8.jpeg" style="height: 50%;"/>

<img src="9.jpeg" style="height: 50%;"/>

<img src="10.jpeg" style="height: 50%;"/>

<img src="11.jpeg" style="height: 50%;"/>

<img src="12.jpeg" style="height: 50%;"/>

<img src="13.jpeg" style="height: 50%;"/>

<img src="14.jpeg" style="height: 50%;"/>

---

# Live Demo of P0 with Exceptions
**Kevin Zhou, Meijing Li, Fanping Jiang**

**COMPSCI 4TB3, McMaster University**

**April 2021**

---

### 1. Import all the dependencies

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from IPython.display import display
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### WebAssembly Exception

some WebAssembly code examples available in the [repo](https://github.com/WebAssembly/wabt/tree/main/test/parse/expr) (try, catch, throw) such as:

```nasm

```

In [ ]:
;;; TOOL: wat2wasm
;;; ARGS: --enable-exceptions
(module
  (tag $e1 (param i32))
  (tag $e2 (param i32 i32))

  ;; multiple catch clauses ending with catch_all
  (func
    try
      nop
    catch $e1
      drop
    catch $e2
      drop
      drop
    catch_all
    end)
)

### Explicit Exception:
- Parsing `throw i` where `i` is integer exception tag

In [ ]:
# "Throw" can be parsed
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 
    then throw 39 
    else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program test
    var a: integer
        a ← sqrt(15)
        write(a)
""")

### Explicit Exception:
- Parsing `try-catch`

In [ ]:
#"try-catch" can be parsed
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 
    then throw 3
    else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program test
    var x, a: integer
    try
        x ← read()
        a ← sqrt(x)
        write(a)
    catch 3
        write(-1)
""", 'trycatch.wat')

In [ ]:
!wat2wasm --enable-exceptions trycatch.wat

In [ ]:
runwasm('trycatch.wasm')

### Explicit Exception:
- Parsing multiple `catch`

In [ ]:
# multiple catch can be parsed
compileString("""
procedure foo(x: integer) → (r: integer)
    if x < 0 then throw 0
    else 
        if x < 10 then throw 10
        else r := x 
program test
    var x, a: integer
    try
        x ← read()
        a ← foo(x)
        write(a)
    catch 0
        write(-1)
    catch 10
        write(5)
""", 'multicatch.wat')

In [ ]:
!wat2wasm --enable-exceptions multicatch.wat

In [ ]:
runwasm('multicatch.wasm')

## Implicit Exception

### Implicit Exception: Index Out of Bound
- Predefined exception keyword: `indexoutofbound`;
- In the implementation level, the keyword `indexoutofbound` matches to the integer exception tag `110`, therefore you can also use `catch 110` and will get the exact same result.

In [ ]:
# Implicit Exception: Index out of bound (exception tag 110)
# == if (i on stack < a.tp.lower) or (i on stack >= a.tp.lower + a.tp.length) then throw else ...
compileString("""
var a: [1..2] → integer
program fool
    var i: integer
    a[1] := 1; a[2] := 2
    try
        i ← read()
        write(a[i])
    catch indexoutofbound
        write(110)
""", 'badindex.wat')

In [ ]:
!wat2wasm --enable-exceptions badindex.wat

In [ ]:
runwasm("badindex.wasm")

### Implicit Exception: Div or Mod by 0
- Div by 0: predefined exception keyword: `zerodiv`. It matches integer exception tag `111`.

In [ ]:
# Implicit Exception: Division by 0 (exception tag 111)
compileString("""
program fool
    var a, x: integer
    try
        x ← read()
        a := 10 div x
        write(a)
    catch zerodiv
        write(111)
""", 'divby0.wat')

In [ ]:
!wat2wasm --enable-exceptions divby0.wat

In [ ]:
runwasm("divby0.wasm")

### Implicit Exception: Div or Mod by 0
- Mod by 0: predefined exception keyword: `zeromod`. It matches integer exception tag `112`.

In [ ]:
# Implicit Exception: Mod by 0 (exception tag 112)
compileString("""
program fool
    var a, x: integer
    try
        x ← read()
        a := 10 mod x
        write(a)
    catch zeromod
        write(112)
""", 'modby0.wat')

In [ ]:
!wat2wasm --enable-exceptions modby0.wat

In [ ]:
runwasm("modby0.wasm")

<img src="15.jpeg" style="height: 50%;"/>

<img src="16.jpeg" style="height: 50%;"/>

<img src="17.jpeg" style="height: 50%;"/>

<img src="18.jpeg" style="height: 50%;"/>

<img src="19.jpeg" style="height: 50%;"/>